# Giskard Robot playground

## 1. Parameters and Helper function

In [1]:
import os
import subprocess
import rospy
from ipywidgets import Button, Layout, GridBox, Box
from IPython.display import display, Markdown, IFrame
from sidecar import Sidecar

LAUNCH_FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))
SIDECAR = None
LAUNCH_PROCESS = None
SELECTED_ROBOT = None

try:
    RIVZWEB = IFrame(src=rospy.get_param('/rvizweb/jupyter_proxy_url'), width='100%', height='100%')
except Exception as e:
    RIVZWEB = None

try:
    SELECTED_ROBOT = rospy.get_param('/nbparam_robot')
except Exception as e:
    SELECTED_ROBOT = None

try:
    HAS_DISPLAY = 'true' if os.environ['DISPLAY'] != ':100' else 'false'
except KeyError:
    HAS_DISPLAY = 'false'

def open_rvizweb():
    global SIDECAR
    if SIDECAR is not None:
        SIDECAR.close()
    if RIVZWEB is not None:
        SIDECAR = Sidecar(title='RVIZWEB', anchor='right')
        with SIDECAR:
            display(RIVZWEB)

def launch_robot(robot):
    global LAUNCH_PROCESS
    if LAUNCH_PROCESS is not None:
        LAUNCH_PROCESS.terminate()
        LAUNCH_PROCESS.wait()
    launchfile = os.path.join(LAUNCH_FILE_DIR, f"{robot}_standalone.launch")
    command = f"roslaunch {launchfile} gui:={HAS_DISPLAY}"
    LAUNCH_PROCESS = subprocess.Popen(['/bin/bash', '-c', command],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL, shell=False)
    open_rvizweb()

def create_button(name):
    btn = Button(
        description=name.upper(),
        layout=Layout(width='auto', height='50px'),
        style={'font_size':'1rem'},
        tooltip=f"Launch robot: {name}"
    )
    btn.on_click(lambda b: launch_robot(name))
    return btn

print(f"Selected Robot: {SELECTED_ROBOT}" if SELECTED_ROBOT is not None else "No robot has been selected yet！")

Selected Robot: stretch


## 2.Launch robot

In [2]:
robot_list = ['pr2', 'tiago', 'hsr', 'donbot', 'tracy', 'boxy', 'stretch', 'armar']
button_widget = GridBox([create_button(robot) for robot in robot_list], layout=Layout(grid_template_columns="repeat(4, 25%)"))
if SELECTED_ROBOT is not None:
    if SELECTED_ROBOT in robot_list:
        launch_robot(SELECTED_ROBOT)
        display(Markdown(f"### Robot **'{SELECTED_ROBOT}'** Launched"))
    else:
        display(Markdown(f"### Robot **'{SELECTED_ROBOT}'** is unavaliable!!!"))
display(Markdown('#### Click the buttons below to launch a new robot.'))
display(button_widget)

### Robot **'stretch'** Launched

#### Click the buttons below to launch a new robot.

GridBox(children=(Button(description='PR2', layout=Layout(height='50px', width='auto'), style=ButtonStyle(font…

## 3. Control the robot with the Giskard Python Interface
> Note: The following code is only for PR2!
### 3.1 Simple example
Reference: https://github.com/SemRoCo/giskardpy/wiki/Old-Python-Interface

In [8]:
import rospy
import random
from giskardpy.python_interface import GiskardWrapper
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped
rospy.init_node('gk_python_demo')
giskard_wrapper = GiskardWrapper()

In [9]:
# list controlled joints
group_name = giskard_wrapper.get_group_names()[0]
giskard_wrapper.get_controlled_joints(group_name)

['brumbrum',
 'head_pan_joint',
 'head_tilt_joint',
 'l_elbow_flex_joint',
 'l_forearm_roll_joint',
 'l_shoulder_lift_joint',
 'l_shoulder_pan_joint',
 'l_upper_arm_roll_joint',
 'l_wrist_flex_joint',
 'l_wrist_roll_joint',
 'r_elbow_flex_joint',
 'r_forearm_roll_joint',
 'r_shoulder_lift_joint',
 'r_shoulder_pan_joint',
 'r_upper_arm_roll_joint',
 'r_wrist_flex_joint',
 'r_wrist_roll_joint',
 'torso_lift_joint']

In [23]:
# Control joint
giskard_wrapper.set_joint_goal({'torso_lift_joint': random.uniform(0.1, 0.4)})
giskard_wrapper.plan_and_execute(wait=True)

error_codes: [0]
error_messages: 
  - None
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
  

In [24]:
# Move robot around
base_goal = PoseStamped()
base_goal.header.frame_id = 'map'
base_goal.pose.position = Point(-2, 1, 0)
base_goal.pose.orientation = Quaternion(0, 0, 0, 1)
giskard_wrapper.set_cart_goal(root_link='map', tip_link='base_footprint', goal_pose=base_goal)
giskard_wrapper.plan_and_execute(wait=True)

error_codes: [0]
error_messages: 
  - None
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
  

### 3.2 Complex example

Source: https://github.com/SemRoCo/giskardpy/blob/master/scripts/examples/python_interface_example.py

In [18]:
import rospy
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped
from giskardpy.python_interface import GiskardWrapper

# create goal joint state dictionary
start_joint_state = {'r_elbow_flex_joint': -1.29610152504,
                     'r_forearm_roll_joint': -0.0301682323805,
                     'r_shoulder_lift_joint': 1.20324921318,
                     'r_shoulder_pan_joint': -0.73456435706,
                     'r_upper_arm_roll_joint': -0.70790051778,
                     'r_wrist_flex_joint': -0.10001,
                     'r_wrist_roll_joint': 0.258268529825,

                     'l_elbow_flex_joint': -1.29610152504,
                     'l_forearm_roll_joint': 0.0301682323805,
                     'l_shoulder_lift_joint': 1.20324921318,
                     'l_shoulder_pan_joint': 0.73456435706,
                     'l_upper_arm_roll_joint': 0.70790051778,
                     'l_wrist_flex_joint': -0.1001,
                     'l_wrist_roll_joint': -0.258268529825,

                     'torso_lift_joint': 0.2,
                     'head_pan_joint': 0,
                     'head_tilt_joint': 0}

# init ros node
# rospy.init_node('test')

rospy.loginfo('Instantiating Giskard wrapper.')
giskard_wrapper = GiskardWrapper()

# Remove everything but the robot.
giskard_wrapper.clear_world()

rospy.loginfo('Combining a joint goal for the arm with a Cartesian goal for the base to reset the pr2.')
# Setting the joint goal
giskard_wrapper.set_joint_goal(start_joint_state)

base_goal = PoseStamped()
base_goal.header.frame_id = 'map'
base_goal.pose.position = Point(0, 0, 0)
base_goal.pose.orientation = Quaternion(0, 0, 0, 1)
# Setting the Cartesian goal.
# Choosing map as root_link will allow Giskard to drive with the pr2.
giskard_wrapper.set_cart_goal(root_link='map', tip_link='base_footprint', goal_pose=base_goal)

# Turn off collision avoidance to make sure that the robot can recover from any state.
giskard_wrapper.allow_all_collisions()
giskard_wrapper.plan_and_execute()

rospy.loginfo('Setting a Cartesian goal for the right gripper.')
r_goal = PoseStamped()
r_goal.header.frame_id = 'r_gripper_tool_frame'
r_goal.pose.position = Point(-0.2, -0.2, 0.2)
r_goal.pose.orientation = Quaternion(0, 0, 0, 1)
giskard_wrapper.set_cart_goal(root_link='map', tip_link='r_gripper_tool_frame', goal_pose=r_goal)

rospy.loginfo('Setting a Cartesian goal for the left gripper.')
l_goal = PoseStamped()
l_goal.header.frame_id = 'l_gripper_tool_frame'
l_goal.pose.position = Point(0.2, 0.2, 0.2)
l_goal.pose.orientation = Quaternion(0, 0, 0, 1)
giskard_wrapper.set_cart_goal(root_link='map', tip_link='l_gripper_tool_frame', goal_pose=l_goal)

rospy.loginfo('Executing both Cartesian goals at the same time')
giskard_wrapper.plan_and_execute()

rospy.loginfo('Combining a Cartesian goal with a partial joint goal.')
p = PoseStamped()
p.header.frame_id = 'map'
p.pose.position.x = 0.8
p.pose.position.y = 0.2
p.pose.position.z = 1.0
p.pose.orientation.w = 1

rospy.loginfo('Setting Cartesian goal.')
# Choosing base_footprint as root_link will not include the base, therefore not allowing pr2 to drive.
giskard_wrapper.set_cart_goal(root_link='base_footprint', tip_link='l_gripper_tool_frame', goal_pose=p)

rospy.loginfo('Setting joint goal for only the torso.')
giskard_wrapper.set_joint_goal({'torso_lift_joint': 0.3})

rospy.loginfo('Executing.')
giskard_wrapper.plan_and_execute()

rospy.loginfo('Setting a pointing goal via the json interface.')
goal_point = PointStamped()
goal_point.header.frame_id = 'r_gripper_tool_frame'

tip = 'high_def_frame'
# The root link torso_lift_link is above the torso joint, therefore the pr2 can't use its torso to achieve the goal.
root = 'torso_lift_link'
pointing_axis = Vector3Stamped()
pointing_axis.header.frame_id = tip
pointing_axis.vector.x = 1

giskard_wrapper.set_json_goal('Pointing',
                              tip_link=tip,
                              root_link=root,
                              goal_point=goal_point,
                              pointing_axis=pointing_axis)
giskard_wrapper.plan_and_execute()

rospy.loginfo('Setting a pointing goal via the predefined Giskard wrapper function to look at the left hand.')
goal_point = PointStamped()
goal_point.header.frame_id = 'l_gripper_tool_frame'

tip = 'high_def_frame'
root = 'torso_lift_link'
pointing_axis = Vector3Stamped()
pointing_axis.header.frame_id = tip
pointing_axis.vector.x = 1

giskard_wrapper.set_pointing_goal(tip_link=tip,
                                  root_link=root,
                                  goal_point=goal_point,
                                  pointing_axis=pointing_axis)

rospy.loginfo('Combining it with a goal that makes the right hand point at the left hand.')
tip = 'r_gripper_tool_frame'
root = 'torso_lift_link'
pointing_axis = Vector3Stamped()
pointing_axis.header.frame_id = tip
pointing_axis.vector.x = 1
giskard_wrapper.set_pointing_goal(tip_link='r_gripper_tool_frame',
                                  root_link=root,
                                  goal_point=goal_point,
                                  pointing_axis=pointing_axis)
rospy.loginfo('Execute')
giskard_wrapper.plan_and_execute()

rospy.loginfo('Spawn a box in the world.')
box_name = 'muh'
box_pose = PoseStamped()
box_pose.header.frame_id = 'r_gripper_tool_frame'
box_pose.pose.orientation.w = 1
giskard_wrapper.add_box(name=box_name,
                        size=(0.2, 0.1, 0.1),
                        pose=box_pose,
                        parent_link='map')
rospy.loginfo('Delete everything but the robot.')
giskard_wrapper.clear_world()

rospy.loginfo('Spawn a box again')
giskard_wrapper.add_box(name=box_name,
                        size=(0.2, 0.1, 0.1),
                        pose=box_pose,
                        parent_link='map')

rospy.loginfo('Attach it to the robot')
giskard_wrapper.update_parent_link_of_group(name=box_name,
                                            parent_link='r_gripper_tool_frame')

rospy.loginfo('Delete only the box.')
giskard_wrapper.remove_group(name=box_name)

rospy.loginfo('Attach a box directly to the robot\'s right gripper.')
giskard_wrapper.add_box(name=box_name,
                        size=(0.2, 0.1, 0.1),
                        pose=box_pose,
                        parent_link='r_gripper_tool_frame')

rospy.loginfo('Set a Cartesian goal for the box')
box_goal = PoseStamped()
box_goal.header.frame_id = box_name
box_goal.pose.position.x = 0.5
box_goal.pose.orientation.w = 1
giskard_wrapper.set_cart_goal(goal_pose=box_goal,
                              tip_link=box_name,
                              root_link='map')
giskard_wrapper.plan_and_execute()

[INFO] [1706022442.976445]: Instantiating Giskard wrapper.
[INFO] [1706022443.503200]: Combining a joint goal for the arm with a Cartesian goal for the base to reset the pr2.
[INFO] [1706022445.348955]: Setting a Cartesian goal for the right gripper.
[INFO] [1706022445.349978]: Setting a Cartesian goal for the left gripper.
[INFO] [1706022445.350724]: Executing both Cartesian goals at the same time
[INFO] [1706022446.580237]: Combining a Cartesian goal with a partial joint goal.
[INFO] [1706022446.581224]: Setting Cartesian goal.
[INFO] [1706022446.581902]: Setting joint goal for only the torso.
[INFO] [1706022446.582471]: Executing.
[INFO] [1706022448.494347]: Setting a pointing goal via the json interface.
[INFO] [1706022449.374957]: Setting a pointing goal via the predefined Giskard wrapper function to look at the left hand.
[INFO] [1706022449.376064]: Combining it with a goal that makes the right hand point at the left hand.
[INFO] [1706022449.376862]: Execute
[INFO] [1706022450.23

error_codes: [0]
error_messages: 
  - None
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
  

## Todos:
- Blockly control
- Fix boxy arms are not visible in Rvizweb
- Armar description not found